In [1]:
import torch
import glob

/home/anhtn/miniconda3/envs/esm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def format_name(protname):
    string_list = protname.split(" ")
    protname_formatted = ""
    notOS = True
    for string in string_list[1:]:
        # while "OS" not in string:
        if "OS" in string:
            notOS = False
        if notOS:
            protname_formatted += string + ' '
    return protname_formatted.strip()

In [3]:
esm_data = {}

In [4]:
for f in glob.glob("esm_data/results/*"):
    if ".pt" not in f:
        tensor_obj = torch.load(glob.glob(f+"/*.pt")[0])
    else:
        tensor_obj = torch.load(f)
    
    protname = format_name(tensor_obj["label"])
    lastlayer_feature = tensor_obj["mean_representations"][33]
    esm_data[protname] = lastlayer_feature.numpy()

In [5]:
esm_data

{'Immunoglobulin lambda constant 7': array([ 0.01818529, -0.07679449, -0.02555966, ..., -0.07907711,
        -0.10127921,  0.04082747], dtype=float32),
 'Putative protein FAM90A16P/FAM90A17P': array([-0.0011677 , -0.11681251,  0.03941101, ..., -0.10838416,
         0.13716166, -0.05012516], dtype=float32),
 'Syntenin-1': array([-0.02560936, -0.0589917 , -0.1134998 , ..., -0.20648947,
        -0.00106089,  0.14154316], dtype=float32),
 'Neurosecretory protein VGF': array([ 0.01863784,  0.03924203, -0.00582737, ..., -0.00071939,
         0.15692519,  0.04466166], dtype=float32),
 'Mitogen-activated protein kinase kinase kinase 7': array([ 0.0220676 , -0.01589068,  0.02207291, ..., -0.15123096,
         0.11835968,  0.10229474], dtype=float32),
 'CD5 antigen-like': array([-0.03373117, -0.07086172,  0.05511188, ..., -0.06693148,
        -0.067843  ,  0.01756851], dtype=float32),
 'Suppression of tumorigenicity 18 protein': array([ 0.00972879, -0.07826491,  0.03326848, ..., -0.05369811,
   

In [21]:
import pandas as pd
corona = pd.read_csv("gt15_csf_plasma_features.csv",header=None)
corona.columns = ["Corona","Protein names"]

In [22]:
corona_name_list = list(corona["Protein names"])
short_name_list = [i.split("(")[0].strip() for i in corona_name_list ]
corona["Short names"] = short_name_list

In [23]:
corona["ESM features"] = corona["Short names"].map(esm_data)
corona[corona["ESM features"].isna()]

,Corona,Protein names,Short names,ESM features
0,0,Serum albumin,Serum albumin,NaN
12,0,Integrin alpha-7 [Cleaved into: Integrin alpha...,Integrin alpha-7 [Cleaved into: Integrin alpha...,NaN
19,1,tRNA-dihydrouridine(47) synthase [NAD(P)(+)]-l...,tRNA-dihydrouridine,NaN
22,1,Protein AMBP [Cleaved into: Alpha-1-microglobu...,Protein AMBP [Cleaved into: Alpha-1-microglobulin,NaN
60,0,Apolipoprotein C-III (Apo-CIII) (ApoC-III) (Ap...,Apolipoprotein C-III,NaN
78,1,Fibrinogen alpha chain [Cleaved into: Fibrinop...,Fibrinogen alpha chain [Cleaved into: Fibrinop...,NaN
87,0,Complement component C9 [Cleaved into: Complem...,Complement component C9 [Cleaved into: Complem...,NaN
112,0,Serum albumin,Serum albumin,NaN
139,0,Neurosecretory protein VGF [Cleaved into: Neur...,Neurosecretory protein VGF [Cleaved into: Neur...,NaN
141,0,Beta-2-microglobulin [Cleaved into: Beta-2-mic...,Beta-2-microglobulin [Cleaved into: Beta-2-mic...,NaN


In [24]:
corona.iloc[0,2] = "Albumin"
corona.iloc[112,2] = "Albumin"
corona.iloc[141,2] = "Beta-2-microglobulin"
corona.iloc[152,2] = "Collagen alpha-2(XI) chain"
corona.iloc[87,2] = "Complement component C9"
corona.iloc[78,2] = "Fibrinogen alpha chain"
corona.iloc[162,2] = "Fibrinogen beta chain"
corona.iloc[165,2] = "Fibrinogen alpha chain"
corona.iloc[12,2] = "Integrin alpha-7"
corona.iloc[139,2] = "Neurosecretory protein VGF"
corona.iloc[22,2] = "Protein AMBP"
corona.iloc[19,2] = "tRNA-dihydrouridine(47) synthase [NAD(P)(+)]-like"

In [25]:
corona["ESM features"] = corona["Short names"].map(esm_data)
corona[corona["ESM features"].isna()]

,Corona,Protein names,Short names,ESM features
60,0,Apolipoprotein C-III (Apo-CIII) (ApoC-III) (Ap...,Apolipoprotein C-III,NaN


# Train set

In [26]:
corona_final = corona.dropna()
corona_final

,Corona,Protein names,Short names,ESM features
0,0,Serum albumin,Albumin,"[-0.050140176, -0.052733008, -0.078817025, 0.0..."
1,1,Haptoglobin (Zonulin) [Cleaved into: Haptoglob...,Haptoglobin,"[0.0064989347, -0.020802459, 0.012509024, 0.00..."
2,1,Immunoglobulin kappa constant (Ig kappa chain ...,Immunoglobulin kappa constant,"[0.057951685, -0.10169613, -0.048828438, 0.163..."
3,1,Immunoglobulin heavy constant gamma 1 (Ig gamm...,Immunoglobulin heavy constant gamma 1,"[0.0092305485, -0.14546646, -0.07357601, 0.218..."
4,0,Serotransferrin (Transferrin) (Beta-1 metal-bi...,Serotransferrin,"[-0.05455521, -0.052603737, -0.06821843, 0.130..."
...,...,...,...,...
169,0,Hemoglobin subunit beta (Beta-globin) (Hemoglo...,Hemoglobin subunit beta,"[-0.012229192, -0.07183029, -0.122249745, 0.03..."
170,0,Haptoglobin (Zonulin) [Cleaved into: Haptoglob...,Haptoglobin,"[0.0064989347, -0.020802459, 0.012509024, 0.00..."
171,0,Immunoglobulin heavy constant alpha 1 (Ig alph...,Immunoglobulin heavy constant alpha 1,"[0.04001641, -0.077039436, -0.009125997, 0.053..."
172,0,Immunoglobulin lambda constant 7 (Ig lambda-7 ...,Immunoglobulin lambda constant 7,"[0.018185293, -0.07679449, -0.02555966, 0.0750..."


In [27]:
corona_final.to_pickle("ESM_train_features_final_original.pkl")

# Test set

In [18]:
test_proteins = ["Transgelin","TAR DNA-binding protein 43","CD44 antigen","Lysozyme C","L-lactate dehydrogenase A chain","Ribonuclease pancreatic","Glutathione S-transferase","Syntenin-1"]
test_prot_dict = {key: esm_data[key] for key in test_proteins}
test_prot_dict

{'Transgelin': array([ 0.01866783, -0.04229837, -0.06194848, ..., -0.08734561,
         0.03154277,  0.08278896], dtype=float32),
 'TAR DNA-binding protein 43': array([ 0.00425845, -0.11756418,  0.01696265, ..., -0.20884526,
         0.08357187,  0.13880682], dtype=float32),
 'CD44 antigen': array([ 0.04093808, -0.09759036, -0.03048422, ..., -0.16888894,
         0.11854924,  0.00402072], dtype=float32),
 'Lysozyme C': array([ 0.00598918, -0.12158216, -0.0545426 , ..., -0.07569703,
        -0.10295825,  0.00341555], dtype=float32),
 'L-lactate dehydrogenase A chain': array([-0.01681066, -0.07554813, -0.0415225 , ..., -0.16483591,
         0.01006651,  0.09918204], dtype=float32),
 'Ribonuclease pancreatic': array([ 0.05048919, -0.09564679, -0.04518586, ..., -0.05201669,
        -0.05163842,  0.04149849], dtype=float32),
 'Glutathione S-transferase': array([ 0.05858101, -0.03697236,  0.0059275 , ..., -0.18125202,
        -0.07133106, -0.06018687], dtype=float32),
 'Syntenin-1': array([-

In [19]:
df = pd.DataFrame(test_proteins)
df.columns = ["Protein names"]
df["ESM features"] = df["Protein names"].map(test_prot_dict)
df

,Protein names,ESM features
0,Transgelin,"[0.01866783, -0.04229837, -0.06194848, 0.03984..."
1,TAR DNA-binding protein 43,"[0.004258453, -0.11756418, 0.016962651, 0.0892..."
2,CD44 antigen,"[0.040938083, -0.097590365, -0.03048422, 0.150..."
3,Lysozyme C,"[0.0059891753, -0.12158216, -0.0545426, 0.0438..."
4,L-lactate dehydrogenase A chain,"[-0.01681066, -0.075548135, -0.041522495, 0.09..."
5,Ribonuclease pancreatic,"[0.050489195, -0.09564679, -0.04518586, 0.1652..."
6,Glutathione S-transferase,"[0.058581013, -0.036972355, 0.0059275, 0.05365..."
7,Syntenin-1,"[-0.025609355, -0.0589917, -0.1134998, 0.09664..."


In [20]:
df.to_pickle("ESM_test_features_final.pkl")